## Selfie Generation

Generates selfies for testing diverse user inputs

In [ ]:
%reload_ext dotenv
%dotenv

In [ ]:
from io import BytesIO
from pathlib import Path

from PIL import Image
from google import genai
from google.genai import types


In [ ]:
client = genai.Client()

prompt = 'regular mirror selfie from reddit, black male wearing tan shirt, no UI'
response = client.models.generate_content(
    model="gemini-3.1-flash-image-preview",
    contents=[prompt],
    config=types.GenerateContentConfig(
        image_config=types.ImageConfig(
            aspect_ratio="3:4",
            image_size="512px",
        ),
    ),
)

assert response.parts is not None

for part in response.parts:
    if part.text is not None:
        print(part.text)
    elif part.inline_data is not None:
        genai_image = part.as_image()
        assert genai_image is not None
        assert genai_image.image_bytes is not None
        image = Image.open(BytesIO(genai_image.image_bytes))

usage = response.usage_metadata
assert usage is not None
print(
    f"\nToken usage: {usage.prompt_token_count} input, {usage.candidates_token_count} output, {usage.total_token_count} total"
)
print(f"Image size: {image.size} px")
preview = image.copy()
preview.thumbnail((600, 600))
preview

In [ ]:
output_dir = Path("../images/humans")
existing = sorted(output_dir.glob("selfie_*.jpg"))
if existing:
    last_n = int(existing[-1].stem.removeprefix("selfie_"))
    n = last_n + 1
else:
    n = 1

output_path = output_dir / f"selfie_{n}.jpg"
image.save(output_path)
print(f"Saved to {output_path}")